# This notebook describes how the XRD-CT data is reconstructed
# Both the 25 micron and 50 micron data will be reconstructed here 
# Phase-based reconstructions will be performed using powder-XRD data as a basis for phase-specific peaks

In [ ]:
# imports

import numpy as np
from numpy import deg2rad, transpose
from h5py import File
from tomopy import recon

In [21]:
# 25 Micron XRD-CT
# Scans 43322, 43323, 43324, 43325

#50 Micron XRD-CT
#Scans 43336 -> 43401

#Example for file 43322
input_file = "k11-43322-diffraction-DiffInt_Mask_Azz_Txt.nxs"
file_num = input_file.split('-')[1].split('_')[0]
output_file_na = f"output_{file_num}_na.nxs"
output_file_zn = f"output_{file_num}_zn.nxs"

In [14]:
# Load the diffraction

with File(input_file) as file:
    key = file["processed/result"]

    data = transpose(key["data"][0],(2,0,1))
    # data exists as an image of 80x40 1D patterns with 5000 q values per 1D diffraction pattern

    q_avg = np.sum(data, axis=(1,2))
    # q_avg sums the intensity for each q_val across all 80x40 1D patterns collected

    q = key["q"][:]

    angles = deg2rad(key["gts_cs_theta"])

In [17]:
# The peaks identified from the powder-XRD data

zn_peaks = [1.101,1.429,1.656,1.675,1.890, 2.068, 2.144, 2.187, 2.258, 2.370, 2.410, 2.625, 2.857]
na_peaks = [1.096, 1.306, 1.423, 1.651, 1.883, 2.059, 2.137, 2.180,2.251, 2.360, 2.401, 2.791, 2.848, 2.881,2.967]

# I want to find the peak values closest to above in q in experimental data

zn = []
zn_idx = []

na = []
na_idx = []

for peak in zn_peaks:
    q_val = min(q, key=lambda x:abs(x-peak))
    zn += [q_val]
    zn_idx += np.where(q==q_val)

for peak in na_peaks:
    q_val = min(q, key=lambda x:abs(x-peak))
    na += [q_val]
    na_idx += np.where(q==q_val)

In [ ]:
# The centre of rotation for the reconstructions needs to be defined
# It corresponds to the centre of the sinogram, since the imaging and diffraction beams are centred to the axis of rotation of the sample

centre_midpoint = 20    # for 25 micron
centre_midpoint = 10    # for 50 micron

In [ ]:
# Average intensity around the peak, and perform reconstrucuing w/gridrec for all Zn and Na peaks

with File(output_file_na, 'w') as file:
    for idx, peak in enumerate(na):
        data_window_na = np.mean(data[int(na_idx[idx])-1:int(na_idx[idx])+1],axis=0,keepdims=True)
        file[f"entry/peak at q~{np.round(na[idx],3)}"] = recon(data_window_na, theta = angles, center = centre_midpoint, sinogram_order = True, algorithm = "gridrec")

with File(output_file_zn, 'w') as file:
    for idx, peak in enumerate(zn):
        data_window_zn = np.mean(data[int(zn_idx[idx])-1:int(zn_idx[idx])+1],axis=0,keepdims=True)
        file[f"entry/peak at q~{np.round(zn[idx],3)}"] = recon(data_window_zn, theta = angles, center = centre_midpoint, sinogram_order = True, algorithm = "gridrec")

# The NeXus file created here (.nxs) can be loaded into DAWN